In [60]:
# 필요한 라이브러리 설치
!pip install -q transformers chromadb langchain-community google-api-python-client huggingface_hub sentence-transformers

In [62]:
from google.colab import userdata

# Colab Secrets에서 인증 정보 로드
#HF_TOKEN = userdata.get('xxxxxxxx') # Hugging Face 액세스 토큰
#GOOGLE_API_KEY = userdata.get('xxxxxxxxxxxx') # Google Custom Search API 키
#GOOGLE_CSE_ID = userdata.get('xxxxxxxxxxxxxxxx') # Programmable Search Engine ID

HF_TOKEN = "xxxxxxxxxxxxxx"
GOOGLE_API_KEY = "xxxxxxxxxxxxxxxxx"
GOOGLE_CSE_ID = "xxxxxxxxxxxxxxx"

In [63]:
# 1. 데이터 수집 및 벡터 저장소 구축 (NIH DSLD 사용)
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [64]:
# 데이터베이스에서 데이터 로드
loader = WebBaseLoader([
    "https://www.healthline.com/nutrition",
    "https://examine.com/supplements/",
    "https://www.iherb.com/c/Magnesium",
    "https://www.iherb.com/c/Vitamin-B12"
    "https://dsld.od.nih.gov/dsld/"
])
documents = loader.load()



In [65]:
# 문서 분할 (청크 크기 512, 50% 오버랩)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=256
)
texts = text_splitter.split_documents(documents)

# 임베딩 모델 초기화 (gte-base 사용)
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-base")

# ChromaDB에 벡터 저장
vector_db = Chroma.from_documents(
    texts, embeddings,
    persist_directory="./chroma_db",
    collection_name="supplement_info"
)

In [66]:
# 2. Google Programmable Search Engine 설정
from googleapiclient.discovery import build

def google_search(query, num=3):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num).execute()
    return [item["snippet"] for item in res["items"]]

# 3. Gemma-3B 모델 초기화
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN
)

In [68]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 1. 로그인 (선택적이지만 확실하게 처리)
login(token="xxxxxxxxxxxxx")  # 토큰 전체 입력

# 2. 모델과 토크나이저 로딩 (토큰 명시)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it", token="xxxxxxxxxxxxxxxxxxxx")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,  # 또는 torch.float32 (Colab CPU 환경이라면)
    token="xxxxxxxxxxxxxxxxxxxxxxxx"
)


In [69]:
# 4. 고급 프롬프트 엔지니어링 템플릿
REACT_PROMPT = """<start_of_turn>user
사용자 정보:
{user_info}

현재 단계: {step}
과거 관측: {history}

다음 단계에서 수행해야 할 작업을 다음 형식 중 하나로 출력:
1. 검색 필요: "Action: search[검색어]"
2. DB 조회: "Action: lookup[질문]"
3. 최종 답변: "Final Answer: [답변]"

규칙:
- 의학적 조언은 공인된 출처를 반드시 인용
- 복용량 계산 시 사용자의 신체 조건 고려
- 상호작용 가능성 경고
<end_of_turn>
<start_of_turn>assistant
"""

In [76]:
def run_agent(user_input, max_steps=5):
    history = []
    for step in range(max_steps):
        # ReAct 프롬프트 구성
        prompt = REACT_PROMPT.format(
            user_info=user_input,
            step=step + 1,
            history="\n".join(history[-3:])
        )

        # 모델 추론
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=500)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # ✅ Step 1 또는 2에서 Final Answer 나오면 무시하고 다음 step 진행
        if step < 2 and "Final Answer:" in response:
            print(f"Step {step + 1}에서 Final Answer가 나와서 무시하고 다음 스텝으로 진행합니다.")
            continue

        # ✅ Final Answer 도출 시 반환
        if "Final Answer:" in response:
            print(f"\nStep {step + 1}: 최종 답변 생성")
            return response.split("Final Answer:")[-1].strip()

        # ✅ 액션 파싱 및 실행
        action_type, query = parse_action(response)
        observation = execute_action(action_type, query)

        print(f"\nStep {step + 1}:")
        print(f"생각: {response.split('Action:')[0].strip()}")
        print(f"액션: {action_type}({query})")
        print(f"관측: {observation[:200]}...")

        # ✅ 관측 결과 저장
        history.append(f"Step {step + 1}: {observation[:200]}")


In [72]:
# # 6. 실행 예시
# user_input = """
# 30대 직장인인데 스트레스가 잘 쌓이고 쉽게 피곤해집니다.
# 잘 지치니 기분도 별로에요. 어떤 영양제 제품을 먹으면 좋을지 추천해주세요.
# 답변은 한글로 작성해주세요.
# """

# result = run_agent(user_input)
# print("\n최종 추천:")
# print(result)

In [77]:
REACT_PROMPT = """<start_of_turn>user
사용자 정보:
{user_info}

현재 단계: {step}
과거 관측: {history}

다음 단계에서 수행해야 할 작업을 다음 형식 중 하나로 출력:
1. 검색 필요: "Action: search[검색어]"
2. DB 조회: "Action: lookup[질문]"
3. 최종 답변: "Final Answer: [답변]"

규칙:
- 반드시 최소 1회 이상 Action: search 또는 Action: lookup을 수행한 후 Final Answer를 출력할 것
- 웹사이트 healthline, examine, iHerb 또는 NIH DSLD에서 근거를 기반으로 할 것
- 답변은 2~4문장 이내로 간결하게 작성
- 핵심 성분을 명확히 언급할 것 (예: 글루코사민, 마그네슘 등)
- 실제 제품 예시와 가격 포함 (예: 닥터스베스트 글루코사민 MSM, 약 25,000원)
- 복용량은 짧게 설명
- 일반 사용자도 쉽게 이해할 수 있도록 한글로 작성
<end_of_turn>
<start_of_turn>assistant
"""


In [78]:
# 6. 실행 예시 (실행 중 사용자에게 입력 받음)
user_input = input("당신의 증상이나 건강 고민을 자유롭게 입력하세요:\n")

result = run_agent(user_input)
print("\n✅ 최종 추천:")
print(result)


당신의 증상이나 건강 고민을 자유롭게 입력하세요:
90세야. 무릎관절아파
Step 1에서 Final Answer가 나와서 무시하고 다음 스텝으로 진행합니다.
Step 2에서 Final Answer가 나와서 무시하고 다음 스텝으로 진행합니다.

Step 3: 최종 답변 생성

✅ 최종 추천:
** 무릎 관절 통증은 다양한 원인으로 발생할 수 있습니다. 특히 노년층의 경우 관절 연골 손상,  퇴행성 관절염, 과도한 체중 등으로 인해 통증이 심해질 수 있습니다. 글루코사민과 마그네슘은 관절 건강에 도움이 될 수 있습니다.
